In [1]:
from JP_stock_parser.yahoo_parser import stockDataReader
from datetime import datetime
from JP_stock_parser.db_model import Record, DBManager, Stock
from datetime import date
import pandas as pd
from tqdm import tqdm
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Update the database to today
with DBManager() as db:
    target_stocks = [item[0] for item in db.query(Stock.stock_id).all()]
    stock_reader = stockDataReader(target_stocks[0])
    for stock in tqdm(target_stocks):
        stock_reader.set_stock(sn=stock)
        stock_reader.update_DB(verbose=False)

100%|██████████| 225/225 [13:53<00:00,  3.70s/it]


In [3]:
# Check the last record for each stock data
with DBManager() as db:
    stocks = [i[0] for i in db.query(Stock.stock_id).all()]
    for stock in stocks:
        record = db.query(Record).filter(Record.stock_id==stock).order_by(Record.record_date.desc()).first()
        print(record)

日水 @2020-02-21
マルハニチロ @2020-02-21
国際石開帝石 @2020-02-21
コムシスＨＤ @2020-02-21
大成建 @2020-02-21
大林組 @2020-02-21
清水建 @2020-02-21
長谷工 @2020-02-21
鹿島 @2020-02-21
ハウス @2020-02-21
積ハウス @2020-02-21
日揮ＨＤ @2020-02-21
日清粉Ｇ @2020-02-21
明治ＨＤ @2020-02-21
日ハム @2020-02-21
エムスリー @2020-02-21
ディーエヌエ @2020-02-21
サッポロＨＤ @2020-02-21
アサヒ @2020-02-21
キリンＨＤ @2020-02-21
宝ＨＬＤ @2020-02-21
双日 @2020-02-21
キッコマン @2020-02-21
味の素 @2020-02-21
ニチレイ @2020-02-21
ＪＴ @2020-02-21
Ｊフロント @2020-02-21
三越伊勢丹 @2020-02-21
東洋紡 @2020-02-21
ユニチカ @2020-02-21
日清紡ＨＤ @2020-02-21
東急不ＨＤ @2020-02-21
セブン＆アイ @2020-02-21
帝人 @2020-02-21
東レ @2020-02-21
クラレ @2020-02-21
旭化成 @2020-02-21
ＳＵＭＣＯ @2020-02-21
王子ＨＤ @2020-02-21
日本紙 @2020-02-21
昭電工 @2020-02-21
住友化 @2020-02-21
日産化 @2020-02-21
東ソー @2020-02-21
トクヤマ @2020-02-21
デンカ @2020-02-21
信越化 @2020-02-21
協和キリン @2020-02-21
三井化学 @2020-02-21
三菱ケミＨＤ @2020-02-21
宇部興 @2020-02-21
日化薬 @2020-02-21
電通グループ @2020-02-21
花王 @2020-02-21
武田 @2020-02-21
アステラス @2020-02-21
大日本住友 @2020-02-21
塩野義 @2020-02-21
中外薬 @2020-02-21
エーザイ @20

In [7]:
# Comibe all the stock data by date and adjclose
# Save the result to csv File


main_df = pd.DataFrame()

with DBManager() as db:
    target_stocks = [item for item in db.query(Stock.stock_id, Stock.name).all()]
    for stock_id, stock_name in tqdm(target_stocks):
        stock_reader = stockDataReader(stock_id)
        df = stock_reader.read_DB()
        df.set_index('date', inplace=True)
        df.rename(columns = {'adjclose': stock_id}, inplace=True)
        df.drop(['open', 'high', 'low', 'close', 'volume'], 1, inplace=True)
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')
print("Finish compling, save to csv")
main_df.to_csv("jp255.csv")

100%|██████████| 225/225 [01:06<00:00,  3.39it/s]


Finish compling, save to csv


In [6]:
main_df.tail(5)

,日水,マルハニチロ,国際石開帝石,長谷工,積ハウス,清水建,コムシスＨＤ,大林組,日揮ＨＤ,鹿島,...,東宝,コナミＨＤ,ＺＨＤ,日本郵政,サイバー,リクルート,セコム,ディーエヌエ,エムスリー,電通グループ
date,,,,,,,,,,,,,,,,,,,,,
2020-02-17,616.0,2571.0,1050.5,1448.0,2408.0,1094.0,2914.0,1237.0,1555.0,1296.0,...,3925.0,4350.0,449.0,1005.5,4220.0,4499.0,9685.0,1604.0,3200.0,3555.0
2020-02-18,617.0,2604.0,1041.5,1426.0,2345.5,1083.0,2911.0,1224.0,1541.0,1297.0,...,3890.0,4290.0,434.0,1011.5,4345.0,4500.0,9521.0,1579.0,3035.0,3545.0
2020-02-19,617.0,2585.0,1042.0,1417.0,2330.0,1092.0,2928.0,1223.0,1558.0,1295.0,...,3895.0,4295.0,430.0,1005.5,4340.0,4553.0,9572.0,1595.0,3075.0,3395.0
2020-02-20,610.0,2565.0,1072.0,1416.0,2317.5,1082.0,2919.0,1229.0,1551.0,1278.0,...,3870.0,4375.0,423.0,1005.0,4410.0,4552.0,9641.0,1573.0,3065.0,3380.0
2020-02-21,594.0,2544.0,1069.5,1408.0,2343.0,1064.0,2855.0,1209.0,1566.0,1260.0,...,3830.0,4335.0,442.0,1003.5,4350.0,4488.0,9502.0,1582.0,3055.0,3325.0
